In [3]:
import json
with open('train/train_split.jsonl',encoding="utf-8") as f:
    train = [json.loads(jline) for jline in f.read().splitlines()]
len(train)

560

In [5]:
from vncorenlp import VnCoreNLP
rdrsegmenter = VnCoreNLP("/home/hoanganh/zac2020-IE/VnCoreNLP/VnCoreNLP-1.1.1.jar", annotators="wseg", max_heap_size='-Xmx500m') 

# Input 
text = "Lê Hoàng Anh"

# To perform word (and sentence) segmentation
sentences = rdrsegmenter.tokenize(text) 
print(sentences)

[['Lê_Hoàng_Anh']]


In [14]:
'sfad fsad. f'.replace(' ','_')

'sfad_fsad._f'

In [ ]:
def find_and_replace(s, sub, replace):
    for i in range(len(s)-len(sub)):
        if sub
for item in train:
    team1 = item['match_summary']['players']['team1']
    for para in item['original_doc']['_source']['body']:
        para.replace(team1,team1.replace(" ",''))

In [26]:
def match_arr(sub, arr, idx):
    ''' Match sub in arr at index idx'''
    for i in range(len(sub)):
        if sub[i]!=arr[idx+i]:
            return False
    return True

def find_array(sub,arr):
    ''' Find first appearance of sub in arr '''
    if len(sub)==0 or len(arr)==0:
        return -1
    for i in range(len(arr)-len(sub)+1):
        if match_arr(sub,arr,i):
            return i
    return -1

def find_all_array(sub,arr):
    ''' Find all appearances of sub in arr '''
    result=[]
    for i in range(len(arr)-len(sub)+1):
        if match_arr(sub,arr,i):
            result.append(i)
    return result

import itertools

def preprocess(text):
    return text.replace('&amp;','&')

def tokenize(text):
    token_sentences = rdrsegmenter.tokenize(preprocess(text))
    return list(itertools.chain.from_iterable(token_sentences))

def item_to_doc(item):
    doc = item['original_doc']['_source']['description']
    for para in item['original_doc']['_source']['body']:
        doc+=' '+para['text']
    return doc

def generate_ner_annotation(item):
    team1, team2 = item['match_summary']['players']['team1'], item['match_summary']['players']['team2']
    team1_token, team2_token = tokenize(team1), tokenize(team2)

    doc_token = tokenize(item_to_doc(item))
    tags = ['O']*len(doc_token)

    team1_pos = find_all_array(team1_token,doc_token)
    for pos in team1_pos:
        tags[pos]='B'+'-'+'team_name'
        for i in range(1,len(team1_token)):
            tags[pos+i]='I'+'-'+'team_name'

    team2_pos = find_all_array(team2_token,doc_token)
    for pos in team2_pos:
        tags[pos]='B'+'-'+'team_name'
        for i in range(1,len(team2_token)):
            tags[pos+i]='I'+'-'+'team_name'

    return list(zip(doc_token,tags))




In [33]:
import codecs
with codecs.open('train/train.txt', 'w', "utf-8") as f:
    for item in train:
        ner = generate_ner_annotation(item)[:256]
        for token in ner:
            f.write(f'{token[0]} {token[1]}\n')
        f.write('\n')

In [32]:
import json
with open('train/val_split.jsonl',encoding="utf-8") as f:
    train = [json.loads(jline) for jline in f.read().splitlines()]

import codecs
with codecs.open('train/dev.txt', 'w', "utf-8") as f:
    for item in train:
        ner = generate_ner_annotation(item)[:256]
        for token in ner:
            f.write(f'{token[0]} {token[1]}\n')
        f.write('\n')

In [31]:
import json,codecs
with open('public_test/public_test.jsonl',encoding="utf-8") as f:
    test = [json.loads(jline) for jline in f.read().splitlines()]

with codecs.open('public_test/test.txt', 'w', "utf-8") as f:
    # json.dump(doc, modified,ensure_ascii=False,indent=4)
    for idx, item in enumerate(test):
        doc_token = tokenize(item_to_doc(item))
        for i,token in enumerate(doc_token):
            if i>=256:
                break
            f.write(f'{token} O\n')
        f.write('\n')